In [1]:
#import Ipynb_importer
#import DbOpModule
import pymysql
import tushare as ts
import numpy as np
import pandas as pd
import time
import sys
import datetime
import json
import ast
import pymongo
import importlib
from pandas.io.json import json_normalize
sys.path.append(r"D:\Jupyter_Pj\pyModule")
import mongodbOp
importlib.reload(mongodbOp)


<module 'mongodbOp' from 'D:\\Jupyter_Pj\\pyModule\\mongodbOp.py'>

In [2]:
db_name = "stock"
daily_col_name = "stock_daily"
stock_basic_col_name = "stock_basic_info"
stock_divid_factors_col_name = "stock_divid_factors"
stock_db_info_name = "stock_db_info"
stock_1mK_col_name = "stock_1mK_line"

daily_start_date = "20120801"
daily_end_date   = "20220811"


In [3]:
pro = ts.pro_api('af5f423bb57f376e71c2903058362f8e3e053fe30f486463f9bdac8e')
#df = ts.pro_bar(ts_code="000001.SZ", adj='qfq', start_date= '20220701' , end_date='20220805')
stockbasic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')
stockbasic
#df = ts.pro_bar(ts_code=sbdf.loc[0,'ts_code'], adj='none', start_date= '20050101' , end_date='20220803')



,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
4861,871981.BJ,871981,晶赛科技,None,None,北交所,20211115
4862,872925.BJ,872925,锦好医疗,None,None,北交所,20211025
4863,873169.BJ,873169,七丰精工,None,None,北交所,20220415
4864,873223.BJ,873223,荣亿精密,None,None,北交所,20220609


In [4]:
db_op = mongodbOp.MongoDbOp()
db_op.connect_To_MongoDb(db_name= db_name)
#stockbasic = db_op.find_any(stock_basic_col_name,{})
#stockbasic
	

In [5]:
def save_stock_daily_to_db():
	for i in stockbasic.index:

		start_time_ = daily_start_date
		if(stockbasic.loc[i,"list_date"] > start_time_):
			start_time_ = stockbasic.loc[i,"list_date"]
		df =pro.daily(ts_code=stockbasic.loc[i,"ts_code"], start_date = start_time_, end_date = daily_end_date)
			#db_op.insert_stock_daily_many(col_name, df)
		df.rename(columns = {"ts_code":"stock_code"},inplace=True)
		db_op.insert_stock_daily_many(daily_col_name, df)
		print ("db_op: insert_stock_daily_many completed!! index = %d, stock_code:= %s, len = %d " %(i,stockbasic.loc[i,"ts_code"],len(df)))

# db_op: insert_stock_daily_many completed!! index = 1888, stock_code:= 300424.SZ, len = 1721 

In [6]:
def save_stock_daily_to_db_2(stockbasic):
	
	df_temp =pro.daily(ts_code="000001.SZ", start_date = "20220811", end_date = "20220811")
	df_temp.rename(columns = {"ts_code":"stock_code"},inplace=True)
	df_temp.drop(df_temp.index, inplace=True)
	i =0 
	for i in stockbasic.index:

		start_time_ = daily_start_date
		if(stockbasic.loc[i,"list_date"] > start_time_):
			start_time_ = stockbasic.loc[i,"list_date"]

		df =pro.daily(ts_code=stockbasic.loc[i,"ts_code"], start_date = start_time_, end_date = daily_end_date)
			#db_op.insert_stock_daily_many(col_name, df)
		print ("db_op: insert_stock_daily_many get daily form tushare website， index : = %d, stock_code:= %s, len = %d" %(i,stockbasic.loc[i,"ts_code"],len(df)))
		
		df.rename(columns = {"ts_code":"stock_code"},inplace=True)

		df_temp = pd.concat([df_temp, df], axis=0, ignore_index=True)

		if( len(df_temp)>= 10000):
			db_op.insert_stock_daily_many(daily_col_name, df_temp)
			print ("db_op: insert_stock_daily_many completed!! index to: = %d, stock_code:= %s, len = %d , size = %d " %(i,stockbasic.loc[i,"ts_code"],len(df_temp), sys.getsizeof(df_temp)))
			df_temp.drop(df_temp.index, inplace=True)
			
		#if( i >=2655):
		#	break	

	if( len(df_temp)>= 1):
		db_op.insert_stock_daily_many(daily_col_name, df_temp)
		print ("db_op: insert_stock_daily_many completed!!  out side !!! index to: = %d, stock_code:= %s, len = %d , size = %d " %(i,stockbasic.loc[i,"ts_code"],len(df_temp), sys.getsizeof(df_temp)))
		df_temp.drop(df_temp.index, inplace=True)
# db_op: insert_stock_daily_many completed!! index = 1888, stock_code:= 300424.SZ, len = 1721 

In [7]:
#save_stock_daily_to_db()

In [8]:
def insert_stock_daily_oneDay(data_):
    df = pro.daily(trade_date=data_)
    db_op.insert_stock_daily_many(daily_col_name, df)

In [9]:
# 获取某个近 100 天的交易日历，从数据库存中获取最近更新数据的日期
# 以交易日为单位获取全部股票的日线信息，并保存

def update_stock_daily_auto():
    now = time.strftime("%Y%m%d")
    start_date = time.strftime("%Y%m%d",time.localtime(time.time() - 100*60*60*24))  #  100 天前的日期
    df = pro.trade_cal(exchange='', start_date = start_date, end_date = now)
    df = df[df["is_open"] ==1]
    df = df.sort_values(by=["cal_date"], ascending=[False])
    df = df.reset_index(drop=True)

    db_val = db_op.find_any(stock_db_info_name, {"_id":"0001"})
    last_update = db_val.stock_daily_update_time.values[0]

    last_update_index = df[df["cal_date"] == last_update].index
    df = df[0:last_update_index[0]]

    for dat in df["cal_date"].values:
        insert_stock_daily_oneDay(dat)
        print ("完全更新 %d 的数据"%(dat) )

    # 更新数据的时间保存到数据库
    info_dict = {"_id":"0001", "stock_daily_update_time":time.strftime("%Y%m%d")}
    db_op.update_record_to_db(stock_db_info_name,info_dict )    
 

In [10]:
#update_stock_daily_auto()

In [11]:
#db_val = db_op.find_any(daily_col_name, {"stock_code" :"600429.SH"})
#db_val

In [12]:
#db_val = db_op.find_any(daily_col_name, {"trade_date" :"20220811","stock_code":"000001.SZ"})
#db_val

In [13]:

info_dict = {"_id":"0001","stock_daily_created":"20220811" ,"stock_daily_update_time":time.strftime("%Y%m%d")}
#db_op.update_record_to_db(stock_db_info_name,info_dict )

In [14]:
db_val = db_op.find_any(stock_db_info_name, {})
db_val

D:\Jupyter_Pj\pyModule\mongodbOp.py:163: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ret)


,_id,stock_daily_update_time,stock_daily_created
0,0001,20220813,20220813


In [34]:
#db_val = db_op.find_any(daily_col_name, {"stock_code" :"000001.SZ"})
#db_val = db_op.find_any(daily_col_name, {"stock_code":"002920.SZ","trade_date":{'$lt':"20220531"}})
#db_val = db_op.find_any(daily_col_name, {"trade_date" :"20220812"})
#db_val = db_op.find_some(daily_col_name, {"stock_code" :"000001.SZ"},ascending = False, num = 10)
#db_val = db_op.find_any(daily_col_name, {})
#db_val = db_op.find_distinct(daily_col_name, "stock_code",{})        
#db_val = db_op.find_distinct(stock_1mK_col_name, "stock_code",{})
db_val = db_op.find_any(stock_1mK_col_name, {"stock_code":"002920.SZ","trade_time":{'$gt':"20220801 09:00:00"}})
#db_val = db_op.find_any(stock_1mK_col_name, {"stock_code" :"002920.SZ"})
db_val

D:\Jupyter_Pj\pyModule\mongodbOp.py:163: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ret)


,_id,amount,close,high,low,open,stock_code,trade_time,volume
0,002920.SZ-20220815 15:00:00,10953828.15,159.98,159.98,159.88,159.88,002920.SZ,20220815 15:00:00,685.0
1,002920.SZ-20220815 14:59:00,0.00,159.88,159.88,159.88,159.88,002920.SZ,20220815 14:59:00,0.0
2,002920.SZ-20220815 14:58:00,0.00,159.88,159.88,159.88,159.88,002920.SZ,20220815 14:58:00,0.0
3,002920.SZ-20220815 14:57:00,6168824.00,159.88,159.91,159.73,159.73,002920.SZ,20220815 14:57:00,386.0
4,002920.SZ-20220815 14:56:00,3529387.00,159.73,159.73,159.65,159.65,002920.SZ,20220815 14:56:00,221.0
...,...,...,...,...,...,...,...,...,...
2646,002920.SZ-20220801 09:34:00,6589270.00,195.36,195.80,195.36,195.55,002920.SZ,20220801 09:34:00,337.0
2647,002920.SZ-20220801 09:33:00,5452603.00,195.50,196.41,193.90,193.90,002920.SZ,20220801 09:33:00,279.0
2648,002920.SZ-20220801 09:32:00,5709878.00,194.20,196.48,194.00,195.24,002920.SZ,20220801 09:32:00,293.0
2649,002920.SZ-20220801 09:31:00,11181277.00,195.96,196.00,192.01,192.10,002920.SZ,20220801 09:31:00,578.0


In [16]:
len(db_val)

6

In [17]:
# 比较tushare 上 stock basic 股票列表与数据库存中某一天的有记录的股票列表 ，整理出新增加的 stock basic 列表， 传给 save_stock_daily_to_db_2
# 如果数据库为空， 则会重新下载的全部记录
def update_diff_stock_daily():
    sb = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,market,list_date')
    db_val = db_op.find_any(daily_col_name, {"trade_date" :"20220812"})
    temp01 = sb["ts_code"].values
    temp02 = set(temp01) -set(db_val)

    sb_val = pd.DataFrame(data = None, columns= sb.columns)
    for sc in temp02:
        df = sb[sb["ts_code"] == sc]
        sb_val = pd.concat([sb_val, df], axis=0, ignore_index=True)

    save_stock_daily_to_db_2(sb_val)

     # 更新数据的时间保存到数据库
    info_dict = {"_id":"0001", "stock_daily_update_time":time.strftime("%Y%m%d")}
    db_op.update_record_to_db(stock_db_info_name,info_dict )

i =3000
start_time_ = stockbasic.loc[i,"list_date"]
df =pro.daily(ts_code=stockbasic.loc[i,"ts_code"], start_date = start_time_, end_date = daily_end_date)
df

In [18]:
#save_stock_daily_to_db_2()